# 1. Data Cleaning and Feature Engineering

This section covers the initial steps of data cleaning and feature engineering.  
The primary goal is to understand the raw data, handle preliminary issues, and create new, more useful features from the existing ones.

## 1.1 Initial Data Inspection and Handling

The first step consisted of a preliminary inspection of the raw dataset.  
To better understand the content and structure of each column, the DataFrame was exported to an Excel file for quick visual inspection.

However, this led to an immediate technical issue.

### Problem: Excel Formula Injection

Some text fields begin with special characters such as `=`, `+`, `-`, or `@`.

When opened in Excel, these are interpreted as formulas, which may corrupt the data or generate errors.

To prevent this issue, a helper function was implemented to prepend a single quote (`'`) to any string starting with these characters, forcing Excel to interpret them as plain text.

In [5]:
import pandas as pd
import numpy as np

file_path = 'data/apple_jobs.csv' 

In [6]:
# 3. Leggere il file CSV e caricarlo in un DataFrame Pandas
try:
    df = pd.read_csv(file_path)
    print("File caricato con successo!")
except FileNotFoundError:
    print(f"Errore: il file '{file_path}' non è stato trovato.")

File caricato con successo!


In [11]:
def escape_excel_formula(x):
    if isinstance(x, str) and x.startswith(('=', '+', '-', '@')):
        return "'" + x
    return x

df_excel = df.map(escape_excel_formula)

# Esportiamo in Excel
output_path = 'data/apple_jobs_EXCEL_visivo.xlsx'
df_excel.to_excel(output_path, index=False)  # index=False evita di esportare l'indice come colonna

print(f"File Excel creato con successo: {output_path}")

File Excel creato con successo: data/apple_jobs_EXCEL_visivo.xlsx


## 1.2 Standardizing Job Titles into Role Categories

The `title` column is a free-text field containing multiple variations of job roles.

The objective is to create a new standardized column called `Role_Category` to group the most common roles into:

- **SD** → Software Developer  
- **SE** → Software Engineer  
- **OTHER** → All remaining roles

### Attempt 1: Strict Matching

The first approach used regular expressions to match full words such as:

- developer
- engineer

Word boundaries (`\b`) were used to ensure exact matching.

In [15]:
# Create lowercase version for matching
df['Title_clean'] = df['title'].str.lower()

cond_sd_v1 = df['Title_clean'].str.contains(r'\bdev(?:eloper)?\b', na=False)
cond_se_v1 = df['Title_clean'].str.contains(r'\beng(?:ineer)?\b', na=False)

df['Role_Category'] = np.select(
    [cond_sd_v1, cond_se_v1],
    ['SD', 'SE'],
    default='OTHER'
)

df['Role_Category'].value_counts()

Role_Category
SE       1380
OTHER     672
SD         96
Name: count, dtype: int64

### Observation

This approach left a high number of roles classified as **OTHER**.

After inspecting these titles, it became clear that many contained abbreviations such as "eng" or partial words like "dev".

### Attempt 2: Flexible Matching

To improve the classification, partial string matching was used instead:

- "dev"
- "eng"

In [17]:
cond_sd_v2 = df['Title_clean'].str.contains(r'dev', na=False)
cond_se_v2 = df['Title_clean'].str.contains(r'eng', na=False)

df['Role_Category'] = np.select(
    [cond_sd_v2, cond_se_v2],
    ['SD', 'SE'],
    default='OTHER'
)

df['Role_Category'].value_counts()

Role_Category
SE       1454
OTHER     488
SD        206
Name: count, dtype: int64

## Reflection

The second approach did not substantially reduce the number of unclassified roles.

However, this raises important analytical questions:

1. Is the SD vs SE distinction truly meaningful?
2. Does the dataset reveal structural differences between these roles?
3. Will this feature uncover meaningful patterns, or is it an oversimplification?

I'm starting to think that my analysis is too generic. Then, what does it really mean to be a developer rather than an engineer? What weight could this distinction carry in our analysis?

## Pure Study of the Variables

Let's pause and take a breath… we are mixing the two methods. In the first analysis, we should **not modify any variable**, but rather understand the meaning of each one before considering the client’s requirements.  

So, if I were to start over, I would use Python code to **visualize the features appropriately and in a readable way**, generating the file **apple_jobs_EXCEL_visivo**. From here, there are two ways to continue:

1. Extract every value of the variable (in this case textual) and have the AI explain in detail what it means. This is because it is difficult to manually analyze almost 2,000 unique texts.  
2. Get a sense of what the variable is trying to represent by sampling only some of the texts.  

I will proceed using the second method first, and then the first one, and compare which approach is more useful by evaluating both speed and efficiency.  

### Manual Description of Variables

Below is a description of the variables:

1. **title:** It is noticeable that the titles start with standard definitions like “Software Engineer,” and then move to more detailed and complex definitions. They are all engineers, but they perform different tasks and have different levels of experience. One could consider subdividing these engineers into more homogeneous macro-categories. For example: front-end software engineers, back-end SE, senior iOS SE, Java SE, etc. To do this without AI, we would need to find keywords for each segmentation (for example: java, iOS, back, front, etc.) and automate the classification by assigning a standard name.  

2. **location:** First, we check how many unique values exist. If there are different cities, the problem is standardizing the nomenclature. We can automate this with a Python function that, each time it encounters a new field, compares it with the others stored in memory (as a vector of words). If it mostly matches one of them (for example, only one or two words differ), it assigns the standard name of that location; otherwise, it stores it in memory as a new value.  

3. **minimum_qual, preferred_qual, responsibilities, education & experience:** These variables are really difficult to standardize into data without using AI. They describe the minimum and preferred requirements for the job, the responsibilities of the employee, and the education and experience required.  

## Objectives of the Work

So, we understand that the dataset concerns job openings in certain specific cities. Now the questions we ask ourselves are:

1. What information can we extract from the data?  
2. Who could benefit from our investigation?  
3. What does the client commissioning the study want to know?  

From the data, we could understand how jobs are distributed geographically, and how the location affects the difficulty and access requirements for a job as a software engineer. For example, this investigation could serve someone looking for a job who doesn’t know where to relocate. Or it could help other companies looking for employees in that area to understand the competition and align themselves. Variables such as proximity to important universities or work centers could also be included. At the moment, no other practical uses for this dataset come to my mind.